In [1]:
from utils.data_preprocessing import *
import os

In [2]:
base_path = os.path.join('..', 'data', 'csv_from_raw_dlis')
csv_data = load_csv_files(base_path)

### Rename the INDEX columns to TDEP

In [3]:
for well, w_dict in csv_data.items():
    for logical_file, lf_dict in w_dict.items():
        for frame, df in lf_dict.items():
            csv_data[well][logical_file][frame] = (
                df.rename(
                    columns=lambda col: 'TDEP' 
                    if col.startswith('INDEX') 
                    else col
                )
            )

### Export the data to CSV files

In [4]:
dataframes_to_csv(csv_data, "../data/dlis_preprocessed")